In [ ]:
#All Downloads go here
# !wget https://cdn.discordapp.com/attachments/750937322299588660/1107618190843990036/medyo.h5

In [ ]:
#!pip install SpeechRecognition pydub pocketsphinx
# %pip install pydub
#!pip install ffmpeg-python
# %pip install ffmpeg

In [ ]:
#All Imports go here
# from IPython.display import Audio, display, HTML
import scipy
import librosa
import numpy as np
import tensorflow as tf
# from google.colab.output import eval_js
# from base64 import b64decode
# from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

In [ ]:
# Set up variables for MFCCs
# words = ['artista', 'sa gawas sa nasud', 'aborsyon', 'katabang', 'hapon', 'sa taas', 'magtatambag', 'nahadlok', 'makaapekto', 'ipahiangay', 'biyaan', 'AIDS', 'kaniadto', 'pagkahuman', 'tambag', 'African-American', 'alyansa', 'makapalit', 'hamtong na', 'kalihokan', 'tinuud', 'tabok', 'abante', 'hingpit nga', 'kaalyado', 'makab-ot', 'moila', 'sa unahan', 'airport', 'aktibista', 'dugangi', 'aktibo', 'moangkon', 'administrasyon', 'edad', 'buhi', 'ayroplano', 'lihok', 'advertising', 'sa tinuud', 'panguma', 'hingpit', 'mouyon', 'advance', 'batok', 'aksidente', 'tumong', 'pag-abuso', 'tabang', 'pagkab-ot', 'ubanan', 'mga nakab-ot', 'Africa', 'mosuhop', 'pagsagop', 'bentaha', 'tin-edyer', 'ahente', 'sa usa ka', 'tigdumala', 'eroplano', 'pagdugang', 'access', 'alkohol', 'sumala', 'pag-usab', 'asoy', 'pagpahiangay', 'makuha', 'manlalaban', 'tanan', 'dugang', 'abilidad', 'dawata', 'ahensya', 'pagdawat', 'agresibo', 'album', 'kasabutan', 'adjust', 'agenda', 'tukma', 'hapit', 'makahimo', 'tugoti', 'asido', 'sumbong', 'ad', 'panimpalad', 'adres', 'akademiko', 'tambagi', 'igo', 'pagkawala', 'aksyon', 'hangin', 'nakadayeg', 'mahitungod sa']
words = ['langit', 'butong', 'ibabaw', 'makapalit', 'lima', 'duha', 'wala', 'sukad', 'asa', 'humok', 'miingon', 'inahan', 'raman', 'dugo', 'ko', 'usba', 'hinungdan', 'busog', 'kasuko', 'giingon', 'saging', 'ulo', 'kaniya', 'tara', 'man', 'pagsulti', 'bati', 'ta', 'kamot', 'uo', 'buntag', 'lugar', 'tiya', 'laay', 'natulog', 'ilaha', 'usab', 'tatay', 'ingato', 'diin', 'basta', 'dugang', 'namo', 'unsa', 'hagad', 'bago', 'tiil', 'unta', 'ate', 'hapon', 'gahapon', 'kasabot', 'dapit', 'kinsa', 'pirmi', 'pito', 'kato', 'asawa', 'tubig', 'nanay', 'iyaha', 'dili', 'hubog', 'kana', 'maulaw', 'mao', 'dako', 'tawo', 'gutom', 'pay', 'hinuon', 'nasod', 'hilantan', 'kani', 'kuya', 'puyo', 'sakyan', 'yuta', 'mangga', 'gwapo', 'aron', 'pila', 'gawas', 'gibuhat', 'buhi', 'pakwan', 'babaye', 'sala', 'kanang', 'siya', 'amahan', 'kwarta', 'amoa', 'nabuang', 'tuo', 'sama', 'kinahanglan', 'sambag', 'kini', 'maayong', 'gani', 'salamat', 'ikaw', 'tiyo', 'gabii', 'bana', 'human', 'gamay', 'ugma', 'gwapa', 'dugay', 'pangutana', 'bukid', 'na', 'mansanas', 'kapayas', 'agi', 'bitaw', 'palihug', 'ako', 'tuig', 'tiguwang', 'buot', 'gahi', 'apan', 'bata', 'ngadto', 'oi', 'pasayloa', 'kita', 'nila', 'ah', 'lalaki', 'libo', 'didto', 'dalan', 'nahurot', 'pero', 'natagak', 'pwede', 'buwan', 'ligo', 'trabaho', 'tambis', 'upat', 'usa', 'maoy', 'ngalan', 'unom', 'balay', 'patay', 'niini', 'among', 'napu', 'amping', 'tungod', 'pinoy', 'ninyo', 'gyod', 'atis', 'unsaon', 'basin', 'kanusa', 'dungan', 'ana', 'abokado', 'atubangan', 'walo', 'may', 'tag', 'tibuok', 'siyam', 'naa', 'istorya', 'nawung', 'balita', 'lingaw', 'naman', 'bantayan', 'naligo', 'bayabas', 'buungon', 'manghuwam', 'luyo', 'unya', 'lang', 'ubos', 'sobra', 'pinaagi', 'tulo', 'kanato', 'nawong', 'hasta', 'lugara', 'buang', 'ang', 'sulod', 'edad', 'balaod', 'iya']
NUM_MFCCS = 13
MAX_LEN = 192
SAMPLE_RATE = 16000
DURATION = 1
HOP_LENGTH = 512
NUM_CLASSES = len(words)

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('weights.h5')

model.summary()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
def pad_mfcc(mfcc, max_len):
    if mfcc.shape[1] > max_len:
        # Trim the MFCCs if the length exceeds the maximum length
        mfcc = mfcc[:, :max_len]
        print("Max Length")
    else:
        # Pad the MFCCs with zeros if the length is less than the maximum length
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc

In [ ]:
# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

In [34]:
def create_chunks(sound_path):
  sound = AudioSegment.from_file(sound_path)
  chunks = split_on_silence (
      # Use the loaded audio.
      sound, 
      # Specify that a silent chunk must be at least 300 milliseconds
      min_silence_len = 150,
      # Consider a chunk silent if it's quieter than 10 dBFS.
      silence_thresh = sound.dBFS - 16,
      keep_silence = 250
  )
  # Process each chunk with your parameters
  for i, chunk in enumerate(chunks):
      # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
      silence_chunk = AudioSegment.silent(duration=00)
      # Add the padding chunk to beginning and end of the entire chunk.
      audio_chunk = silence_chunk + chunk + silence_chunk
      # Normalize the entire chunk.
      normalized_chunk = match_target_amplitude(audio_chunk, -25.0)
      # Export the audio chunk with new bitrate.
      print("Exporting chunk{0}.wav.".format(i))
      normalized_chunk.export(
          ".//chunk{0}.wav".format(i),
          bitrate = "256k",
          format = "wav"
      )
      !ffmpeg -i "chunk{i}.wav" -af silenceremove=1:0:-33dB -ac 1 -ar 16000 -vn "cleaned-chunk{i}.wav" -y -loglevel panic
      #os.system(f'ffmpeg -i "chunk'+ str(i) +'.wav" -af silenceremove=1:0:-33dB -ac 1 -ar 16000 -vn "cleaned/chunk'+ str(i) +'" -y')
  return len(chunks)

In [ ]:
#this is to test the segmentation
def test_chunks(file_path):

  count = create_chunks(file_path)
  predictions = []

  for i in range(count):
    audio_segment, sr = librosa.load("cleaned-chunk{0}.wav".format(i), sr=SAMPLE_RATE)
    mfccs = librosa.feature.mfcc(y=audio_segment, sr=sr, n_mfcc=NUM_MFCCS)
    mfcc_padded = pad_mfcc(mfccs, MAX_LEN)
    prediction = model.predict(mfcc_padded.reshape(1, NUM_MFCCS, MAX_LEN, 1))
    predicted_class = np.argmax(prediction)
    predictions.append(predicted_class)

  predicted_words = [words[pred] for pred in predictions]
  text = ''
  for word in predicted_words:
      text += word + ' '
  print(text)

In [ ]:
#this is for testing singular words
def test_single(test_path):
  test_audio, sr = librosa.load(test_path, sr=SAMPLE_RATE)
  test_mfcc = librosa.feature.mfcc(y=test_audio, sr=sr, n_mfcc=NUM_MFCCS)
  q = model.predict(pad_mfcc(test_mfcc, MAX_LEN).reshape(1, NUM_MFCCS, MAX_LEN, 1))
  print(words[np.argmax(q)])
  print(np.argmax(q))

In [36]:
!rm.exe cleaned-chunk*.wav
test_chunks("ay_lmao.wav")
!rm.exe -v chunk*.wav

Exporting chunk0.wav.
Exporting chunk1.wav.
Exporting chunk2.wav.
Exporting chunk3.wav.
Exporting chunk4.wav.
1/1 [==============================] - 0s 19ms/step
ta duha ulo ta sama 
removed 'chunk0.wav'
removed 'chunk1.wav'
removed 'chunk2.wav'
removed 'chunk3.wav'
removed 'chunk4.wav'


In [19]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wavfile
import google_api as ga
import gpt_api_tagalog as gt

def record_audio(segment_duration=1.0, silence_threshold=30):
    # Configure audio recording parameters
    sample_rate = 44100  # Sample rate (Hz)

    # Initialize variables
    segment = np.array([], dtype=np.float32)
    silent_duration = 40
    silent_frames = 0
    state = 0
    def audio_callback(indata, frames, time, status):
        nonlocal segment, silent_frames, state

        volume_norm = np.linalg.norm(indata)*10
        # print("volume: {0} || silent_frames: {1}".format(int(volume_norm), silent_frames))
        # print(rms)

        if volume_norm < silence_threshold:
            silent_frames += 1
            if state == 0:
                segment = np.array([], dtype=np.float32)
        else:
            silent_frames = 0
            state = 1

        # Append the current audio segment to the list if it contains enough samples
        segment = np.concatenate([segment, indata[:, 0]])
        if silent_frames >= silent_duration and state == 1:
            segment = float2pcm(segment)
            save_audio_segment(segment)
            # print("Segment Saved")
            state = 0
            segment = np.array([], dtype=np.float32)

    # Start audio recording
    with sd.InputStream(callback=audio_callback, channels=1, samplerate=sample_rate, device=1):
        print("Recording audio... Press Ctrl+C to stop.")
        while True:
            try:
                sd.sleep(100)
                if silent_frames >= sample_rate * segment_duration:
                    break
            except KeyboardInterrupt:
                break
    return

def save_audio_segment(segment):
    filename = f"D:\\subtlr\\ay_lmao.wav"
    wavfile.write(filename, 44100, segment)
    try:
        transcript = ga.transcribe("D:\\subtlr\\ay_lmao.wav")
        print("Transcript: {}".format(gt.translate(transcript)))
        test_chunks("ay_lmao.wav")
    except:
        pass
    finally:
        pass
        # !rm.exe -v chunk*.wav
        # !rm.exe cleaned-chunk*.wav


def float2pcm(sig, dtype='int16'):
    sig = np.asarray(sig)
    dtype = np.dtype(dtype)
    i = np.iinfo(dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig * abs_max + offset).clip(i.min, i.max).astype(dtype)

In [20]:
record_audio()

Recording audio... Press Ctrl+C to stop.
Recorded Text: duha tulo
Transcript:  Two three.
Exporting chunk0.wav.
Exporting chunk1.wav.
Exporting chunk2.wav.
1/1 [==============================] - 0s 18ms/step
ta gahapon kamot 
